<a href="https://colab.research.google.com/github/bezerraluis/Projeto-AgriDecision/blob/main/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Próximos passos
- Fazer um repositório com mais imagens

In [1]:
!git clone https://github.com/starly/Soybean_Images_MachineLearning

Cloning into 'Soybean_Images_MachineLearning'...
remote: Enumerating objects: 11, done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 11
Unpacking objects: 100% (11/11), done.


In [2]:
cd Soybean_Images_MachineLearning/

/content/Soybean_Images_MachineLearning


In [3]:
ls

README.md                Soybean_healthy_test.zip   Soybean_rust_train.zip
Soybean_downy_test.zip   Soybean_healthy_train.zip
Soybean_downy_train.zip  Soybean_rust_test.zip


In [4]:
!unzip /content/Soybean_Images_MachineLearning/Soybean_downy_train.zip
!unzip /content/Soybean_Images_MachineLearning/Soybean_rust_train.zip
!unzip /content/Soybean_Images_MachineLearning/Soybean_healthy_train.zip

Archive:  /content/Soybean_Images_MachineLearning/Soybean_downy_train.zip
  inflating: Soybean_downy_train/Agron_Iastate_downymildewleaf_001.jpg  
  inflating: Soybean_downy_train/Downey_mildew_undel_2.PNG  
  inflating: Soybean_downy_train/downy_mildew_1_thesoyreport.jpg  
  inflating: Soybean_downy_train/downy_mildew_2_thesoyreport.jpg  
  inflating: Soybean_downy_train/downy_mildew_IL_lg_soybeanresearchinfo.jpg  
  inflating: Soybean_downy_train/Downy_Mildew_Soybean_Arkansas.PNG  
  inflating: Soybean_downy_train/Downy_Mildew_Soybean_Arkansas2.PNG  
  inflating: Soybean_downy_train/Downy_Mildew_undel.PNG  
  inflating: Soybean_downy_train/downy_mildew_university_of_minnesota_dmild2_200px.jpg  
  inflating: Soybean_downy_train/insectimages_powdery_mildew5469487.jpg  
  inflating: Soybean_downy_train/university_of_minnesota_dmild3_200px.jpg  
  inflating: Soybean_downy_train/university_of_minnesota_dmild4_200px.jpg  
  inflating: Soybean_downy_train/University_of_Minnesota_downy_milde

In [8]:
!rm -r /content/Soybean_Images_MachineLearning/.git

In [17]:
import glob

filepaths = glob.glob('/content/Soybean_Images_MachineLearning/*/*.{jpg,jpeg,png}')

In [18]:
sum('rust_train' in f for f in filepaths)

0

In [13]:
sum('healthy_train' in f for f in filepaths)

15

In [14]:
sum('downy_train' in f for f in filepaths)

10

In [16]:
len(filepaths)

112

In [37]:
# TODO: Import the model and the preprocess_input function
from keras.applications.resnet50 import ResNet50, preprocess_input

# TODO: Import the ImageDataGenerator class
from keras.preprocessing.image import ImageDataGenerator

# Shape in which all images are going to be reshaped
TARGET_SHAPE = (224, 224, 3)

# TODO: Initialize the data generator class 
datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             validation_split=0.2)

# TODO: Create the training and validation generators using the method flow_from_directory
train_gen = datagen.flow_from_directory('/content/Soybean_Images_MachineLearning', 
                                        target_size=TARGET_SHAPE[:2], 
                                        class_mode='sparse',
                                        subset='training')

valid_gen = datagen.flow_from_directory('/content/Soybean_Images_MachineLearning', 
                                        target_size=TARGET_SHAPE[:2], 
                                        class_mode='sparse',
                                        subset='validation')

Found 114 images belonging to 3 classes.
Found 27 images belonging to 3 classes.


In [33]:
sum(train_gen.classes==1)

40

In [34]:
sum(train_gen.classes==0)

14

In [35]:
sum(train_gen.classes==2)

87

In [27]:
train_gen.class_indices

{'Soybean_downy_train': 0, 'Soybean_healthy_train': 1, 'Soybean_rust_train': 2}

In [38]:
# 1. TODO: Import any additional packages
from keras.models import Sequential
from keras.layers import Flatten, Dense, GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50

# 2. TODO: Initialize base model
base_model = ResNet50(include_top=False, input_shape=TARGET_SHAPE)

# 3. TODO: Freeze layers from the base model
for layer in base_model.layers:
    layer.trainable=False
    
# 4. TODO: Add Fully connected layer to the base model
model = Sequential([base_model,
                    GlobalAveragePooling2D(),
                    Dense(1024, activation='relu'),
                    Dense(3, activation='softmax')])

In [39]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=1e-4), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [41]:
model.fit_generator(train_gen, train_gen.n//train_gen.batch_size, epochs=3,
                    validation_data=valid_gen)

Epoch 1/3
3/3 [==============================] - 15s 5s/step - loss: 0.2694 - accuracy: 0.9062 - val_loss: 0.4585 - val_accuracy: 0.8148
Epoch 2/3
3/3 [==============================] - 14s 5s/step - loss: 0.2274 - accuracy: 0.9167 - val_loss: 0.4073 - val_accuracy: 0.8519
Epoch 3/3
3/3 [==============================] - 14s 5s/step - loss: 0.1448 - accuracy: 0.9583 - val_loss: 0.3768 - val_accuracy: 0.8519


In [30]:
model.fit_generator?